Equal-Weight S&P 500 Index Fund

Introduction

The S&P 500 is the world's most popular stock market index. 
The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. 
It has more than US$250 billion of assets under management.

[own S&P 500 Index Fund <=> own 500 largest companies in the U.S]

The goal is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

In [25]:
import os
import math
import requests # HTTP request
#import xlswriter
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
from tqdm import tqdm
import pandas_datareader.data as web_dt_rdr

Scrape S&P Ticker Symbol by Web Scrapping (from Google WIKI)

In [23]:
sp500_companies_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_ticker_symbol_table = pd.read_html (sp500_companies_url)

# Since WIKI has multiple tables, select the relevant table
# [in this case the first table, index = 0]
display (sp500_ticker_symbol_table [0])
display (type (sp500_ticker_symbol_table [0]))

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


pandas.core.frame.DataFrame

In [20]:
ticker_symbol_extracted = sp500_ticker_symbol_table [0]['Symbol']
ticker_symbol_extracted

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     YUM
499    ZBRA
500     ZBH
501    ZION
502     ZTS
Name: Symbol, Length: 503, dtype: object

Scrape Stocks Data (from Yahoo)

In [37]:
today = dt.datetime.today().date()
print (today)

#df = web_dt_rdr.DataReader (ticker_symbol_extracted.to_list(), 'yahoo', today)

for each_ticker in ticker_symbol_extracted :
     display (each_ticker)
     df = web_dt_rdr.DataReader (each_ticker, 'yahoo', today)

disply(df)

2023-01-06


'MMM'

TypeError: string indices must be integers

API Set-Up
A dummy cell to test API is working correctly

In [11]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
#print (IEX_CLOUD_API_TOKEN)

symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
print (api_url)
try :
    api_resp = requests.get (api_url) # <-- Response Object : <Response [200]> / <Response [404]> 
    data = api_resp.json() # Response Object --> json
    #print (type(data))
    for key in data.keys() :
        print (f"{key} : {data [key]}")
        print ()
except Exception as e:
        print (f"WARNING !!! error loading --[{symbol}]-- : {e}")

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448
WARNING !!! error loading --[AAPL]-- : Expecting value: line 1 column 1 (char 0)


Creating DataFrame of Stocks Data

In [12]:
api_query_result = []

for each_stock in tqdm(stocks ['Ticker'], position=0, leave=True) :
    #print (each_stock) # uncomment for debugging
    try : 
        api_url = f"https://sandbox.iexapis.com/stable/stock/{each_stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
        api_resp = requests.get (api_url)
        data = api_resp.json()
        api_query_result.append ([each_stock, data ['latestPrice'], data ['marketCap'], 'N/A'])
    except Exception as e:
        print (f"WARNING !!! error loading --[{each_stock}]-- : {e}")
        pass

#print (api_query_result) # uncomment for debugging

  0%|                                           | 1/505 [00:00<07:23,  1.14it/s]

WARNING !!! error loading --[A]-- : Expecting value: line 1 column 1 (char 0)


  0%|▏                                          | 2/505 [00:01<07:04,  1.18it/s]

WARNING !!! error loading --[AAL]-- : Expecting value: line 1 column 1 (char 0)


  1%|▎                                          | 3/505 [00:02<07:00,  1.20it/s]

...

WARNING !!! error loading --[AIV]-- : Expecting value: line 1 column 1 (char 0)


  4%|█▋                                        | 21/505 [00:17<06:36,  1.22it/s]

WARNING !!! error loading --[AIZ]-- : Expecting value: line 1 column 1 (char 0)


  4%|█▋                                        | 21/505 [00:17<06:53,  1.17it/s]


KeyboardInterrupt: 

In [1]:
stocks_df = pd.DataFrame (api_query_result, columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']) 
stocks_df

NameError: name 'pd' is not defined

In [ ]:
Market Cap => The value of the company = Number of Outstanding Shares * Price